In [1]:
# !pip install sklearn --upgrade
# !pip install joblib
# !pip install imbalanced-learn --upgrade
# !pip install mlxtend  

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from imblearn.over_sampling import (RandomOverSampler, SMOTE, ADASYN)
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
import itertools


Using TensorFlow backend.


In [2]:
# Read the data
nd = pd.read_csv("data.csv")
numeric_data = nd.drop(columns="Unnamed: 32")
numeric_data.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [3]:
numeric_data['diagnosis_num'] =  numeric_data['diagnosis'].apply(lambda x: 0 if x == 'B' else 1)
numeric_data.columns
numeric_data = numeric_data[['id', 'diagnosis', 'diagnosis_num', 'radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst']]

numeric_data.head(15)

,id,diagnosis,diagnosis_num,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,1,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.26540,0.4601,0.11890
1,842517,M,1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.18600,0.2750,0.08902
2,84300903,M,1,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.24300,0.3613,0.08758
3,84348301,M,1,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.25750,0.6638,0.17300
4,84358402,M,1,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.16250,0.2364,0.07678
5,843786,M,1,12.45,15.70,82.57,477.1,0.12780,0.17000,0.15780,...,15.47,23.75,103.40,741.6,0.1791,0.5249,0.5355,0.17410,0.3985,0.12440
6,844359,M,1,18.25,19.98,119.60,1040.0,0.09463,0.10900,0.11270,...,22.88,27.66,153.20,1606.0,0.1442,0.2576,0.3784,0.19320,0.3063,0.08368
7,84458202,M,1,13.71,20.83,90.20,577.9,0.11890,0.16450,0.09366,...,17.06,28.14,110.60,897.0,0.1654,0.3682,0.2678,0.15560,0.3196,0.11510
8,844981,M,1,13.00,21.82,87.50,519.8,0.12730,0.19320,0.18590,...,15.49,30.73,106.20,739.3,0.1703,0.5401,0.5390,0.20600,0.4378,0.10720
9,84501001,M,1,12.46,24.04,83.97,475.9,0.11860,0.23960,0.22730,...,15.09,40.68,97.65,711.4,0.1853,1.0580,1.1050,0.22100,0.4366,0.20750


In [4]:
numeric_only = numeric_data.drop('diagnosis', axis=1)
numeric_df = pd.DataFrame(numeric_only)
numeric_df.head()

,id,diagnosis_num,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,1,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,1,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,1,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,1,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [5]:
#
target = numeric_df['diagnosis_num']
data = numeric_df.drop('diagnosis_num', axis=1)
features = data.columns

In [6]:
# EDA 
total_count_tumors = len(numeric_data)
print(f'Total Tumors on data = {total_count_tumors}')
total_count_benign = (numeric_data.diagnosis == 'B').sum()
print(f'Total Benign Tumors on data = {total_count_benign}')
total_count_malignant = (numeric_data.diagnosis == 'M').sum()
print(f'Total Malignant Tumors on data = {total_count_malignant}')
print("-----")
percent_benign = (total_count_benign/total_count_tumors)*100.00
percent_malignant = (total_count_malignant/total_count_tumors)*100.00
print(f"Percent Benign = {percent_benign}")
print(f"Percent Malignant = {percent_malignant}")

Total Tumors on data = 569
Total Benign Tumors on data = 357
Total Malignant Tumors on data = 212
-----
Percent Benign = 62.741652021089635
Percent Malignant = 37.258347978910365


# Features Selection

Select the top 10 features according to the ```feature_selection``` from ```sklearn```

In [7]:
bestfeatures = SelectKBest(k=31)
fit = bestfeatures.fit(data, target)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(data.columns)
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Feature','Score']
print(featureScores.nlargest(10,'Score'))

                 Feature       Score
28  concave points_worst  964.385393
23       perimeter_worst  897.944219
8    concave points_mean  861.676020
21          radius_worst  860.781707
3         perimeter_mean  697.235272
24            area_worst  661.600206
1            radius_mean  646.981021
4              area_mean  573.060747
7         concavity_mean  533.793126
27       concavity_worst  436.691939


In [8]:
#select the top 10 features
features_list_df = numeric_data[["concave points_worst",
"perimeter_worst",
"concave points_mean",
"radius_worst",
"perimeter_mean",
"area_worst",
"radius_mean",
"area_mean",
"concavity_mean",
"concavity_worst"]]

## OVERSAMPLING THE DATA

In [9]:
# spliting the data on train and test data
X_train, X_test, y_train, y_test = train_test_split (features_list_df, target, test_size = 0.30, random_state=21)
print (X_train.shape, y_train.shape)
print (X_test.shape, y_test.shape)

(398, 10) (398,)
(171, 10) (171,)


In [10]:
# Count the training data
total_count_tumors = len(y_train)
print(f'Total Tumors on train data = {total_count_tumors}')
total_count_benign = (y_train == 0).sum()
print(f'Total Benign Tumors on train data = {total_count_benign}')
total_count_malignant = (y_train == 1).sum()
print(f'Total Malignant Tumors on train data = {total_count_malignant}')
print("-----")
percent_benign = (total_count_benign/total_count_tumors)*100.00
percent_malignant = (total_count_malignant/total_count_tumors)*100.00
print(percent_benign)
print(percent_malignant)

Total Tumors on train data = 398
Total Benign Tumors on train data = 250
Total Malignant Tumors on train data = 148
-----
62.8140703517588
37.185929648241206


In [11]:
print('Original train dataset shape %s' % Counter(y_train))

Original train dataset shape Counter({0: 250, 1: 148})


#### Apply oversampling models (RandomOverSampler, SMOTE, ADASYN)

In [12]:
# Random Over Sampler Model
ros = RandomOverSampler(random_state=21)
X_ros, y_ros = ros.fit_resample(X_train, y_train)
print('Resampled RandomOverSampler train dataset shape %s' % Counter(y_ros))
print (X_ros.shape, y_ros.shape)

Resampled RandomOverSampler train dataset shape Counter({0: 250, 1: 250})
(500, 10) (500,)


In [13]:
# SMOTE: Synthetic Minority Oversampling Technique Model
sm = SMOTE(random_state=21)
X_sm, y_sm = sm.fit_resample(X_train, y_train)
print('Resampled SMOTE train dataset shape %s' % Counter(y_sm))
print (X_sm.shape, y_sm.shape)

Resampled SMOTE train dataset shape Counter({0: 250, 1: 250})
(500, 10) (500,)


In [14]:
# ADASYN: Adaptive Synthetic Sampling Model
ada = ADASYN(random_state=21)
X_ada, y_ada = ada.fit_resample(X_train, y_train)
print('Resampled ADASYN train dataset shape %s' % Counter(y_ada))
print (X_ada.shape, y_ada.shape)

Resampled ADASYN train dataset shape Counter({1: 254, 0: 250})
(504, 10) (504,)


Oversampling models comparison SMOTE VS ADASYN.

SMOTE: It finds the n-nearest neighbors in the minority class for each of the samples in the class. Then it draws a line between the the neighbors an generates random points on the lines.

ADASYN:  Works as SMOTE, but with a minor improved. After creating the sample it adds a random small values to the points. In other words instead of all the sample being linearly correlated to the parent they have a little more variance in them, been a bit scattered.

To take advantage of this smaller variation on the new data, we will use the ADASYN model to oversample the data (X_ada, y_ada).

## Pre-processing

Scale the data using the ```StandardScaler```



In [15]:
# Scale the data
X_scaler = StandardScaler().fit(X_ada)
X_train_scaled = X_scaler.transform(X_ada)
X_test_scaled = X_scaler.transform(X_test)
X_total_scaled = X_scaler.transform(features_list_df)


# Machine Learn Models

### Model 1 - Logistic Regression

In [16]:
# import and train the model
from sklearn import linear_model

logreg = linear_model.LogisticRegression()
logreg.fit(X_train_scaled, y_ada)
prediction_logReg = logreg.predict(X_test_scaled)

In [17]:
print(f"Logistic Regression Training Data Score: {logreg.score(X_train_scaled, y_ada)}")
print(f"Logistic Regression Testing Data Score: {logreg.score(X_test_scaled, y_test)}\n")

print(classification_report(y_test, prediction_logReg,
                            target_names=['Benign ', 'Malignant']))

Logistic Regression Training Data Score: 0.9345238095238095
Logistic Regression Testing Data Score: 0.9298245614035088

              precision    recall  f1-score   support

     Benign        0.97      0.92      0.94       107
   Malignant       0.87      0.95      0.91        64

    accuracy                           0.93       171
   macro avg       0.92      0.93      0.93       171
weighted avg       0.93      0.93      0.93       171



In [18]:
# use GridSearchCV to Hyperparameter Tuning the model

param_grid_logreg = {'C': [5, 10, 50, 100],"penalty": ['l1', 'l2']}
grid_logreg = GridSearchCV(logreg, param_grid_logreg, verbose=3)
grid_logreg.fit(X_train_scaled, y_ada)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] C=5, penalty=l1 .................................................
[CV] ....................... C=5, penalty=l1, score=nan, total=   0.0s
[CV] C=5, penalty=l1 .................................................
[CV] ....................... C=5, penalty=l1, score=nan, total=   0.0s
[CV] C=5, penalty=l1 .................................................
[CV] ....................... C=5, penalty=l1, score=nan, total=   0.0s
[CV] C=5, penalty=l1 .................................................
[CV] ....................... C=5, penalty=l1, score=nan, total=   0.0s
[CV] C=5, penalty=l1 .................................................
[CV] ....................... C=5, penalty=l1, score=nan, total=   0.0s
[CV] C=5, penalty=l2 .................................................
[CV] ..................... C=5, penalty=l2, score=0.921, total=   0.0s
[CV] C=5, penalty=l2 .................................................
[CV] ............

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\joaog\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
C:\Users\joaog\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)
C:\Users\joaog\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit fa


[CV] C=50, penalty=l2 ................................................
[CV] .................... C=50, penalty=l2, score=0.931, total=   0.0s
[CV] C=50, penalty=l2 ................................................
[CV] .................... C=50, penalty=l2, score=0.950, total=   0.0s
[CV] C=50, penalty=l2 ................................................
[CV] .................... C=50, penalty=l2, score=0.941, total=   0.0s
[CV] C=50, penalty=l2 ................................................
[CV] .................... C=50, penalty=l2, score=0.880, total=   0.0s
[CV] C=100, penalty=l1 ...............................................
[CV] ..................... C=100, penalty=l1, score=nan, total=   0.0s
[CV] C=100, penalty=l1 ...............................................
[CV] ..................... C=100, penalty=l1, score=nan, total=   0.0s
[CV] C=100, penalty=l1 ...............................................
[CV] ..................... C=100, penalty=l1, score=nan, total=   0.0s
[CV] 

C:\Users\joaog\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)
C:\Users\joaog\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\joaog\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL

GridSearchCV(cv=None, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [5, 10, 50, 100], 'penalty': ['l1', 'l2']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=3)

In [19]:
print(f"Best Logistic Regression Parameters {grid_logreg.best_params_}")
print(f"Best Logistic Regression Result {grid_logreg.best_score_}")

Best Logistic Regression Parameters {'C': 5, 'penalty': 'l2'}
Best Logistic Regression Result 0.9264950495049504


In [20]:
# set the best Logistic Regression model
bestLogReg_model = linear_model.LogisticRegression(C=5, penalty="l2")

In [21]:
# apply the model 
bestLogReg_model.fit(X_train_scaled, y_ada)
print(f"Best Logistic Regression Model Testing Data Score: {bestLogReg_model.score(X_test_scaled, y_test)}")

Best Logistic Regression Model Testing Data Score: 0.935672514619883


### Model 2 - SVC


In [22]:
# import, train and apply the model
from sklearn.svm import SVC 

model_SVC = SVC()
model_SVC.probability = True
model_SVC.fit(X_train_scaled, y_ada)
prediction_SVC = model_SVC.predict(X_test_scaled)

In [23]:
print(f"SVC Training Data Score: {model_SVC.score(X_train_scaled, y_ada)}")
print(f"SVC Testing Data Score: {model_SVC.score(X_test_scaled, y_test)}\n")
print(classification_report(y_test, prediction_SVC,
                            target_names=['Benign ', 'Malignant']))

SVC Training Data Score: 0.9325396825396826
SVC Testing Data Score: 0.9649122807017544

              precision    recall  f1-score   support

     Benign        0.97      0.97      0.97       107
   Malignant       0.95      0.95      0.95        64

    accuracy                           0.96       171
   macro avg       0.96      0.96      0.96       171
weighted avg       0.96      0.96      0.96       171



In [24]:
# use GridSearchCV to Hyperparameter Tuning the model

param_grid_SVC = {'C': [1, 5, 10, 50, 100],
              'gamma': [0.0001, 0.0005, 0.001, 0.005],
                 "kernel": ["linear",'rbf']}
grid_SVC = GridSearchCV(model_SVC, param_grid_SVC, verbose=3)

In [25]:
grid_SVC.fit(X_train_scaled, y_ada)

Fitting 5 folds for each of 40 candidates, totalling 200 fits
[CV] C=1, gamma=0.0001, kernel=linear ................................
[CV] .... C=1, gamma=0.0001, kernel=linear, score=0.911, total=   0.0s
[CV] C=1, gamma=0.0001, kernel=linear ................................
[CV] .... C=1, gamma=0.0001, kernel=linear, score=0.941, total=   0.0s
[CV] C=1, gamma=0.0001, kernel=linear ................................
[CV] .... C=1, gamma=0.0001, kernel=linear, score=0.960, total=   0.0s
[CV] C=1, gamma=0.0001, kernel=linear ................................
[CV] .... C=1, gamma=0.0001, kernel=linear, score=0.960, total=   0.0s
[CV] C=1, gamma=0.0001, kernel=linear ................................
[CV] .... C=1, gamma=0.0001, kernel=linear, score=0.870, total=   0.0s
[CV] C=1, gamma=0.0001, kernel=rbf ...................................
[CV] ....... C=1, gamma=0.0001, kernel=rbf, score=0.871, total=   0.0s
[CV] C=1, gamma=0.0001, kernel=rbf ...................................
[CV] ....... C=

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


[CV] ....... C=1, gamma=0.0001, kernel=rbf, score=0.610, total=   0.0s
[CV] C=1, gamma=0.0005, kernel=linear ................................
[CV] .... C=1, gamma=0.0005, kernel=linear, score=0.911, total=   0.0s
[CV] C=1, gamma=0.0005, kernel=linear ................................
[CV] .... C=1, gamma=0.0005, kernel=linear, score=0.941, total=   0.0s
[CV] C=1, gamma=0.0005, kernel=linear ................................
[CV] .... C=1, gamma=0.0005, kernel=linear, score=0.960, total=   0.0s
[CV] C=1, gamma=0.0005, kernel=linear ................................
[CV] .... C=1, gamma=0.0005, kernel=linear, score=0.960, total=   0.0s
[CV] C=1, gamma=0.0005, kernel=linear ................................
[CV] .... C=1, gamma=0.0005, kernel=linear, score=0.870, total=   0.0s
[CV] C=1, gamma=0.0005, kernel=rbf ...................................
[CV] ....... C=1, gamma=0.0005, kernel=rbf, score=0.941, total=   0.0s
[CV] C=1, gamma=0.0005, kernel=rbf ...................................
[CV] .

[CV] ........ C=5, gamma=0.005, kernel=rbf, score=0.870, total=   0.0s
[CV] C=10, gamma=0.0001, kernel=linear ...............................
[CV] ... C=10, gamma=0.0001, kernel=linear, score=0.911, total=   0.0s
[CV] C=10, gamma=0.0001, kernel=linear ...............................
[CV] ... C=10, gamma=0.0001, kernel=linear, score=0.941, total=   0.0s
[CV] C=10, gamma=0.0001, kernel=linear ...............................
[CV] ... C=10, gamma=0.0001, kernel=linear, score=0.970, total=   0.0s
[CV] C=10, gamma=0.0001, kernel=linear ...............................
[CV] ... C=10, gamma=0.0001, kernel=linear, score=0.941, total=   0.0s
[CV] C=10, gamma=0.0001, kernel=linear ...............................
[CV] ... C=10, gamma=0.0001, kernel=linear, score=0.880, total=   0.0s
[CV] C=10, gamma=0.0001, kernel=rbf ..................................
[CV] ...... C=10, gamma=0.0001, kernel=rbf, score=0.921, total=   0.0s
[CV] C=10, gamma=0.0001, kernel=rbf ..................................
[CV] .

[CV] ...... C=50, gamma=0.0005, kernel=rbf, score=0.870, total=   0.0s
[CV] C=50, gamma=0.001, kernel=linear ................................
[CV] .... C=50, gamma=0.001, kernel=linear, score=0.911, total=   0.1s
[CV] C=50, gamma=0.001, kernel=linear ................................
[CV] .... C=50, gamma=0.001, kernel=linear, score=0.941, total=   0.0s
[CV] C=50, gamma=0.001, kernel=linear ................................
[CV] .... C=50, gamma=0.001, kernel=linear, score=0.970, total=   0.0s
[CV] C=50, gamma=0.001, kernel=linear ................................
[CV] .... C=50, gamma=0.001, kernel=linear, score=0.941, total=   0.1s
[CV] C=50, gamma=0.001, kernel=linear ................................
[CV] .... C=50, gamma=0.001, kernel=linear, score=0.880, total=   0.0s
[CV] C=50, gamma=0.001, kernel=rbf ...................................
[CV] ....... C=50, gamma=0.001, kernel=rbf, score=0.921, total=   0.0s
[CV] C=50, gamma=0.001, kernel=rbf ...................................
[CV] .

[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    5.1s finished


GridSearchCV(cv=None, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=True, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [1, 5, 10, 50, 100],
                         'gamma': [0.0001, 0.0005, 0.001, 0.005],
                         'kernel': ['linear', 'rbf']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=3)

In [26]:
print(f"Best SVC Parameters {grid_SVC.best_params_}")
print(f"Best SVC Result {grid_SVC.best_score_}")

Best SVC Parameters {'C': 100, 'gamma': 0.005, 'kernel': 'rbf'}
Best SVC Result 0.9304554455445544


In [27]:
# set the best SCV model
bestSVC_model = SVC(C=100, kernel='rbf', gamma=0.005)
bestSVC_model.probability = True

In [28]:
#Best SVC model results
bestSVC_model.fit(X_train_scaled, y_ada)
print(f"Best SVC Model Testing Data Score: {bestSVC_model.score(X_test_scaled, y_test)}")

Best SVC Model Testing Data Score: 0.9473684210526315


### Model 3 - Neural Network

In [29]:
# import, train and apply the model
from sklearn.neural_network import MLPClassifier

nn_model = MLPClassifier()
nn_model.fit(X_train_scaled, y_ada)
prediction_nn = nn_model.predict(X_test_scaled)

C:\Users\joaog\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [30]:
print(f"Neural Network Training Data Score: {nn_model.score(X_train_scaled, y_ada)}")
print(f"Neural Network Testing Data Score: {nn_model.score(X_test_scaled, y_test)}\n")

print(classification_report(y_test, prediction_nn,
                            target_names=['Benign ', 'Malignant']))

Neural Network Training Data Score: 0.9503968253968254
Neural Network Testing Data Score: 0.9590643274853801

              precision    recall  f1-score   support

     Benign        0.97      0.96      0.97       107
   Malignant       0.94      0.95      0.95        64

    accuracy                           0.96       171
   macro avg       0.96      0.96      0.96       171
weighted avg       0.96      0.96      0.96       171



In [31]:
# use GridSearchCV to Hyperparameter Tuning the model

param_grid_nn = {'hidden_layer_sizes': [1,3,5,9,13,18,20],
              "activation":["identity", "logistic", "tanh", "relu"],
                 "solver":["lbfgs", "sgd", "adam"],
                }
grid_nn = GridSearchCV(nn_model, param_grid_nn, verbose=3,cv=3,
                           scoring='accuracy')

In [32]:
grid_nn.fit(X_train_scaled, y_ada)

Fitting 3 folds for each of 84 candidates, totalling 252 fits
[CV] activation=identity, hidden_layer_sizes=1, solver=lbfgs .........
[CV]  activation=identity, hidden_layer_sizes=1, solver=lbfgs, score=0.911, total=   0.0s
[CV] activation=identity, hidden_layer_sizes=1, solver=lbfgs .........
[CV]  activation=identity, hidden_layer_sizes=1, solver=lbfgs, score=0.940, total=   0.0s
[CV] activation=identity, hidden_layer_sizes=1, solver=lbfgs .........
[CV]  activation=identity, hidden_layer_sizes=1, solver=lbfgs, score=0.905, total=   0.0s
[CV] activation=identity, hidden_layer_sizes=1, solver=sgd ...........


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
C:\Users\joaog\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\joaog\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


[CV]  activation=identity, hidden_layer_sizes=1, solver=sgd, score=0.940, total=   0.1s
[CV] activation=identity, hidden_layer_sizes=1, solver=sgd ...........
[CV]  activation=identity, hidden_layer_sizes=1, solver=sgd, score=0.911, total=   0.1s
[CV] activation=identity, hidden_layer_sizes=1, solver=sgd ...........


C:\Users\joaog\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\joaog\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


[CV]  activation=identity, hidden_layer_sizes=1, solver=sgd, score=0.893, total=   0.1s
[CV] activation=identity, hidden_layer_sizes=1, solver=adam ..........
[CV]  activation=identity, hidden_layer_sizes=1, solver=adam, score=0.643, total=   0.1s
[CV] activation=identity, hidden_layer_sizes=1, solver=adam ..........


...

[CV]  activation=relu, hidden_layer_sizes=20, solver=sgd, score=0.851, total=   0.2s
[CV] activation=relu, hidden_layer_sizes=20, solver=adam .............
[CV]  activation=relu, hidden_layer_sizes=20, solver=adam, score=0.911, total=   0.2s
[CV] activation=relu, hidden_layer_sizes=20, solver=adam .............


C:\Users\joaog\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


[CV]  activation=relu, hidden_layer_sizes=20, solver=adam, score=0.946, total=   0.2s
[CV] activation=relu, hidden_layer_sizes=20, solver=adam .............
[CV]  activation=relu, hidden_layer_sizes=20, solver=adam, score=0.905, total=   0.2s


C:\Users\joaog\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
[Parallel(n_jobs=1)]: Done 252 out of 252 | elapsed:   26.8s finished


GridSearchCV(cv=3, error_score=nan,
             estimator=MLPClassifier(activation='relu', alpha=0.0001,
                                     batch_size='auto', beta_1=0.9,
                                     beta_2=0.999, early_stopping=False,
                                     epsilon=1e-08, hidden_layer_sizes=(100,),
                                     learning_rate='constant',
                                     learning_rate_init=0.001, max_fun=15000,
                                     max_iter=200, momentum=0.9,
                                     n_iter_no_change=10,
                                     nesterovs_momentum=True, power_t=0.5,
                                     random_state...e, shuffle=True,
                                     solver='adam', tol=0.0001,
                                     validation_fraction=0.1, verbose=False,
                                     warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={

In [33]:
print(f"Best Neural Network Parameters {grid_nn.best_params_}")
print(f"Best Neural Network Result {grid_nn.best_score_}")

Best Neural Network Parameters {'activation': 'tanh', 'hidden_layer_sizes': 20, 'solver': 'lbfgs'}
Best Neural Network Result 0.9503968253968255


In [34]:
# set the best Neural Network model
bestnn_model = MLPClassifier(activation= 'relu', hidden_layer_sizes= 20, solver= 'lbfgs')

In [35]:
#Best Neural Network model results
bestnn_model.fit(X_train_scaled, y_ada)
print(f"Best Neural Network Model Testing Data Score: {bestnn_model.score(X_test_scaled, y_test)}")

Best Neural Network Model Testing Data Score: 0.9298245614035088


### Model 4 - Random Forest

In [36]:
# import the model 
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rf.fit(X_train_scaled, y_ada)
prediction_rf = rf.predict(X_test_scaled)

In [37]:
print(f"RandonForest Training Data Score: {rf.score(X_train_scaled, y_ada)}")
print(f"RandonForest Testing Data Score: {rf.score(X_test_scaled, y_test)}\n")

print(classification_report(y_test, prediction_rf,
                            target_names=['Benign ', 'Malignant']))

RandonForest Training Data Score: 1.0
RandonForest Testing Data Score: 0.9473684210526315

              precision    recall  f1-score   support

     Benign        0.96      0.95      0.96       107
   Malignant       0.92      0.94      0.93        64

    accuracy                           0.95       171
   macro avg       0.94      0.95      0.94       171
weighted avg       0.95      0.95      0.95       171



In [38]:
# use GridSearchCV to Hyperparameter Tuning the model

param_grid_rf = {'n_estimators': [250, 300, 350]
                 ,'max_depth': [125, 150, 175]}
grid_rf = GridSearchCV(rf, param_grid_rf, verbose=3)

In [39]:
grid_rf.fit(X_train_scaled, y_ada)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] max_depth=125, n_estimators=250 .................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ..... max_depth=125, n_estimators=250, score=0.931, total=   0.3s
[CV] max_depth=125, n_estimators=250 .................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s


[CV] ..... max_depth=125, n_estimators=250, score=0.941, total=   0.3s
[CV] max_depth=125, n_estimators=250 .................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.4s remaining:    0.0s


[CV] ..... max_depth=125, n_estimators=250, score=0.950, total=   0.3s
[CV] max_depth=125, n_estimators=250 .................................
[CV] ..... max_depth=125, n_estimators=250, score=0.931, total=   0.3s
[CV] max_depth=125, n_estimators=250 .................................
[CV] ..... max_depth=125, n_estimators=250, score=0.880, total=   0.3s
[CV] max_depth=125, n_estimators=300 .................................
[CV] ..... max_depth=125, n_estimators=300, score=0.931, total=   0.3s
[CV] max_depth=125, n_estimators=300 .................................
[CV] ..... max_depth=125, n_estimators=300, score=0.950, total=   0.3s
[CV] max_depth=125, n_estimators=300 .................................
[CV] ..... max_depth=125, n_estimators=300, score=0.950, total=   0.3s
[CV] max_depth=125, n_estimators=300 .................................
[CV] ..... max_depth=125, n_estimators=300, score=0.941, total=   0.3s
[CV] max_depth=125, n_estimators=300 .................................
[CV] .

[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:   14.0s finished


GridSearchCV(cv=None, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              ra

In [40]:
print(f"Best RandomForest Parameters {grid_rf.best_params_}")
print(f"Best RandomForest result {grid_rf.best_score_}")

Best RandomForest Parameters {'max_depth': 150, 'n_estimators': 350}
Best RandomForest result 0.9324356435643564


In [41]:
# set the best RandomForest model
best_rf = RandomForestClassifier(max_depth=150, n_estimators=350)
best_rf.probability = True

In [42]:
best_rf.fit(X_train_scaled, y_ada)
print(f"Best RandomForest Model Testing Data Score: {best_rf.score(X_test_scaled, y_test)}")

Best RandomForest Model Testing Data Score: 0.9532163742690059


### Model 5 - Gradient Boost

In [43]:
# import the model 
from sklearn.ensemble import GradientBoostingClassifier

grad_clf = GradientBoostingClassifier()
grad_clf.fit(X_train_scaled, y_ada)
prediction_grad = grad_clf.predict(X_test_scaled)


In [44]:
print(f"GradientBoosting Training Data Score: {grad_clf.score(X_train_scaled, y_ada)}")
print(f"GradientBoosting Testing Data Score: {grad_clf.score(X_test_scaled, y_test)}")

print(classification_report(y_test, prediction_grad,
                            target_names=['Benign ', 'Malignant']))

GradientBoosting Training Data Score: 1.0
GradientBoosting Testing Data Score: 0.9415204678362573
              precision    recall  f1-score   support

     Benign        0.95      0.95      0.95       107
   Malignant       0.92      0.92      0.92        64

    accuracy                           0.94       171
   macro avg       0.94      0.94      0.94       171
weighted avg       0.94      0.94      0.94       171



In [45]:
# use GridSearchCV to Hyperparameter Tuning the model

param_grid_grd = {'n_estimators': [100, 250, 300, 350],'max_depth': [3, 50, 100, 150, 200]}
grid_grad = GridSearchCV(grad_clf, param_grid_grd, verbose=3)
grid_grad.fit(X_train_scaled, y_ada)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV] max_depth=3, n_estimators=100 ...................................
[CV] ....... max_depth=3, n_estimators=100, score=0.911, total=   0.1s
[CV] max_depth=3, n_estimators=100 ...................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV] ....... max_depth=3, n_estimators=100, score=0.950, total=   0.1s
[CV] max_depth=3, n_estimators=100 ...................................
[CV] ....... max_depth=3, n_estimators=100, score=0.941, total=   0.1s
[CV] max_depth=3, n_estimators=100 ...................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.2s remaining:    0.0s


[CV] ....... max_depth=3, n_estimators=100, score=0.931, total=   0.1s
[CV] max_depth=3, n_estimators=100 ...................................
[CV] ....... max_depth=3, n_estimators=100, score=0.900, total=   0.1s
[CV] max_depth=3, n_estimators=250 ...................................
[CV] ....... max_depth=3, n_estimators=250, score=0.911, total=   0.3s
[CV] max_depth=3, n_estimators=250 ...................................
[CV] ....... max_depth=3, n_estimators=250, score=0.960, total=   0.3s
[CV] max_depth=3, n_estimators=250 ...................................
[CV] ....... max_depth=3, n_estimators=250, score=0.941, total=   0.3s
[CV] max_depth=3, n_estimators=250 ...................................
[CV] ....... max_depth=3, n_estimators=250, score=0.931, total=   0.3s
[CV] max_depth=3, n_estimators=250 ...................................
[CV] ....... max_depth=3, n_estimators=250, score=0.900, total=   0.3s
[CV] max_depth=3, n_estimators=300 ...................................
[CV] .

[CV] ..... max_depth=150, n_estimators=100, score=0.941, total=   0.2s
[CV] max_depth=150, n_estimators=100 .................................
[CV] ..... max_depth=150, n_estimators=100, score=0.921, total=   0.2s
[CV] max_depth=150, n_estimators=100 .................................
[CV] ..... max_depth=150, n_estimators=100, score=0.901, total=   0.2s
[CV] max_depth=150, n_estimators=100 .................................
[CV] ..... max_depth=150, n_estimators=100, score=0.900, total=   0.2s
[CV] max_depth=150, n_estimators=250 .................................
[CV] ..... max_depth=150, n_estimators=250, score=0.911, total=   0.3s
[CV] max_depth=150, n_estimators=250 .................................
[CV] ..... max_depth=150, n_estimators=250, score=0.931, total=   0.2s
[CV] max_depth=150, n_estimators=250 .................................
[CV] ..... max_depth=150, n_estimators=250, score=0.921, total=   0.2s
[CV] max_depth=150, n_estimators=250 .................................
[CV] .

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   24.4s finished


GridSearchCV(cv=None, error_score=nan,
             estimator=GradientBoostingClassifier(ccp_alpha=0.0,
                                                  criterion='friedman_mse',
                                                  init=None, learning_rate=0.1,
                                                  loss='deviance', max_depth=3,
                                                  max_features=None,
                                                  max_leaf_nodes=None,
                                                  min_impurity_decrease=0.0,
                                                  min_impurity_split=None,
                                                  min_samples_leaf=1,
                                                  min_samples_split=2,
                                                  min_weight_fraction_leaf=0.0,
                                                  n_estimators=100,
                                                  n_iter_no_change=None,
      

In [46]:
print(f"Best GradientBoosting Parameters {grid_grad.best_params_}")
print(f"Best GradientBoosting result {grid_grad.best_score_}")

Best GradientBoosting Parameters {'max_depth': 3, 'n_estimators': 350}
Best GradientBoosting result 0.9324752475247525


In [47]:
# set the best GradientBoosting model
bestgrad_model = GradientBoostingClassifier(max_depth=3,n_estimators=350)

In [48]:
#Best SVC model results
bestgrad_model.fit(X_train_scaled, y_ada)
print(f"Best GradientBoosting Model Testing Data Score: {bestgrad_model.score(X_test_scaled, y_test)}")

Best GradientBoosting Model Testing Data Score: 0.9415204678362573


## Ensemble a better Model

### Test the tuned models accuracies with ```crossvalidation```

In [49]:
from sklearn import model_selection

clf1 = bestLogReg_model
clf2 = bestSVC_model
clf3 = bestnn_model
clf4 = best_rf
clf5 = bestgrad_model


labels = ["Logistic Regression", 'SVC', "Neural Network",'Random Forest', 'GradientBoosting']

for clf, label in zip([clf1, clf2, clf3, clf4, clf5], labels):

        scores = model_selection.cross_val_score(clf, X_test_scaled, y_test, scoring='accuracy')
        predicted = model_selection.cross_val_predict(clf,X_test_scaled, y_test)
        diff = predicted - y_test
        misclass_indexes = diff[diff != 0].index.tolist()
        print(f"{label} Model - Accuracy: {scores.mean():0.3f} (+/- {scores.std():0.3f})\nMissclassed data index {misclass_indexes}\n")


Logistic Regression Model - Accuracy: 0.930 (+/- 0.029)
Missclassed data index [347, 13, 476, 508, 421, 152, 385, 491, 213, 414, 40, 484]

SVC Model - Accuracy: 0.936 (+/- 0.033)
Missclassed data index [13, 242, 476, 421, 152, 385, 126, 489, 414, 40, 484]

Neural Network Model - Accuracy: 0.936 (+/- 0.033)
Missclassed data index [13, 421, 119, 385, 491, 489, 414, 209, 40, 484]

Random Forest Model - Accuracy: 0.942 (+/- 0.026)
Missclassed data index [13, 476, 385, 536, 126, 491, 414, 209, 40, 484]

GradientBoosting Model - Accuracy: 0.924 (+/- 0.023)
Missclassed data index [13, 476, 152, 385, 126, 491, 489, 91, 213, 414, 209, 40, 469, 484]



### Combining the models to ensemble an improved model

#### Giving different weight to models based on least missclassed data

In [52]:

from mlxtend.classifier import EnsembleVoteClassifier
import copy
eclf = EnsembleVoteClassifier(clfs=[clf1, clf2, clf3, clf4, clf5], weights=[2,2,2,2,1], refit=False)
eclf.fit(X_test_scaled, y_test)

print('Ensemble Model accuracy:', np.mean(y_test == eclf.predict(X_test_scaled))*100, "%")

Ensemble Model accuracy: 95.90643274853801 %


## The Final ensemble model gave us an accuracy of 95.90%.